In [28]:
import numpy as np
from pandas import DataFrame
import pandas as pd
import scipy as sp
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn import svm

Reading data:

In [29]:
def read_data(folder, name1, name2, name3):
    loc = "%s/%s.txt" %(folder, name1)
    train_set = pd.read_csv(loc, sep="\t")
    print "Training set has %s rows and %s coumns" %(train_set.shape[0], train_set.shape[1])
    loc = "%s/%s.txt" %(folder, name2)
    test_set = pd.read_csv(loc, sep="\t")
    print "Testing set has %s rows and %s coumns" %(test_set.shape[0], test_set.shape[1])
    loc = "%s/%s.txt" %(folder, name3)
    val_set = pd.read_csv(loc, sep="\t")
    print "Validation set has %s rows and %s coumns" %(val_set.shape[0], val_set.shape[1])
    return train_set, test_set, val_set



In [30]:
train_set, test_set, val_set = read_data("data5", "train_data", "test_data", "valid_data")

Training set has 428 rows and 25 coumns
Testing set has 142 rows and 25 coumns
Validation set has 142 rows and 25 coumns


In [42]:
train_set_np = train_set.values
test_set_np = test_set.values
val_set_np = val_set.values

In [44]:
def np_cleaner(train, test, val):
    train = train[~np.isnan(train).any(axis=1)]
    print "Training set has %s rows and %s coumns" %(train.shape[0], train.shape[1])
    test = test[~np.isnan(test).any(axis=1)]
    print "Testing set has %s rows and %s coumns" %(test.shape[0], test.shape[1])
    val = val[~np.isnan(val).any(axis=1)]
    print "Validation set has %s rows and %s coumns" %(val.shape[0], val.shape[1])
    return train, test, val

In [56]:
train_set_nb, test_set_nb, val_set_nb = np_cleaner(train_set_np, test_set_np, val_set_np)

Training set has 428 rows and 25 coumns
Testing set has 142 rows and 25 coumns
Validation set has 142 rows and 25 coumns


In [52]:
train_set.columns

Index([u'GP', u'GS', u'MIN', u'FGM', u'FGA', u'FG%', u'3PM', u'3PA', u'3P%',
       u'FTM', u'FTA', u'FT%', u'OFF', u'DEF', u'TRB', u'AST', u'STL', u'BLK',
       u'PF', u'TOV', u'PTS', u'YR', u'POS', u'W', u'H'],
      dtype='object')

In [48]:
columns = {"GP":0, "GS":1, "MIN":2, "FGM":3,"FGA":4,"FG%":5,"3PM":6,"3PA":7,"3P%":8,"FTM":9,"FTA":10,"FT%":11,"OFF":12,"DEF":13,
              "TRB":14,"AST":15,"STL":16,"BLK":17,"PF":18,"TOV":19,"PTS":20,"YR":21,"POS":22,"W":23,"H":24} 

In [53]:
def np_labeliser(train, test, val, col):
    train = train[:,col]
    test = test[:,col]
    val = val[:,col]
    return train, test, val

In [54]:
 train_labels, test_labels, val_labels = np_labeliser(train_set_np, test_set_np, val_set_np, 22)

array([ 4.,  4.,  4.,  5.,  5.,  4.,  1.,  5.,  3.,  1.,  3.,  2.,  5.,
        3.,  4.,  2.,  5.,  5.,  3.,  1.,  1.,  1.,  3.,  4.,  3.,  4.,
        3.,  5.,  5.,  5.,  1.,  5.,  1.,  1.,  4.,  3.,  3.,  3.,  4.,
        5.,  4.,  5.,  5.,  4.,  1.,  4.,  5.,  5.,  4.,  4.,  4.,  5.,
        3.,  2.,  5.,  2.,  2.,  5.,  4.,  5.,  4.,  5.,  5.,  5.,  4.,
        4.,  5.,  5.,  5.,  4.,  2.,  2.,  4.,  3.,  2.,  5.,  2.,  2.,
        5.,  3.,  5.,  4.,  2.,  5.,  2.,  5.,  2.,  1.,  3.,  5.,  3.,
        5.,  5.,  5.,  5.,  2.,  4.,  3.,  3.,  5.,  1.,  5.,  1.,  5.,
        1.,  5.,  4.,  5.,  1.,  3.,  5.,  4.,  3.,  3.,  5.,  4.,  4.,
        5.,  4.,  1.,  4.,  1.,  2.,  4.,  1.,  5.,  4.,  4.,  5.,  5.,
        5.,  1.,  3.,  5.,  2.,  5.,  5.,  3.,  3.,  5.,  1.,  4.,  2.,
        5.,  2.,  2.,  3.,  5.,  2.,  5.,  5.,  5.,  4.,  2.,  1.,  5.,
        2.,  4.,  4.,  4.,  4.,  5.,  3.,  4.,  5.,  2.,  5.,  3.,  3.,
        5.,  5.,  4.,  5.,  4.,  3.,  4.,  4.,  1.,  5.,  2.,  2